In [1]:
import torch

print(torch.__version__)

2.5.1


In [13]:
pip install torch_geometric

Note: you may need to restart the kernel to use updated packages.


# Классификация графов с использованием графовых нейронных сетей

Нам предоставляется набор данных с множеством графов и, в данном примере, необходимо их линейно разделить:

![linear_separability](linear_separability.png)

Классификация графов часто используется в задаче предсказания свойств молекул по её форме, представленной в виде графа, и необходимо определить, подавляет ли молекула ВИЧ или нет. 

В данной работе используется датасет MUTAG. MUTAG Dataset — это коллекция нитроароматических соединений, цель которой — предсказать их мутагенность на Salmonella typhimurium. В наборе представлены химические соединения в виде графов, где вершины соответствуют атомам, а рёбра между вершинами — связям между соответствующими атомами. Импортируем его и считаем характеристики:

In [2]:
import torch
from torch_geometric.datasets import TUDataset

dataset = TUDataset(root='data/TUDataset', name='MUTAG')

print(f'Датасет : {dataset}')
print('=============================')
print(f'Количество графов: {len(dataset)}')
print(f'Количество классов: {dataset.num_classes}')


data = dataset[0]
print()
print(f'Первый граф: {data}')
print('=============================')

# Информация о первом графе
print(f'Количество узлов: {data.num_nodes}')
print(f'Количество ребер: {data.num_edges}')
print(f'Среднее количество рёбер у узла: {data.num_edges / data.num_nodes:.2f}')
print(f'Количество атрибутов узлов: {data.num_node_features}')
print(f'Количество атрибутов рёбер: {data.num_edge_features}')
print(f'Есть изолированные узлы: {data.has_isolated_nodes()}')
print(f'Есть циклы: {data.has_self_loops()}')
print(f'Направленный: {data.is_directed()}')

Датасет : MUTAG(188)
Количество графов: 188
Количество классов: 2

Первый граф: Data(edge_index=[2, 38], x=[17, 7], edge_attr=[38, 4], y=[1])
Количество узлов: 17
Количество ребер: 38
Среднее количество рёбер у узла: 2.24
Количество атрибутов узлов: 7
Количество атрибутов рёбер: 4
Есть изолированные узлы: False
Есть циклы: False
Направленный: False


В данном датасете у нас имеется 188 разных графов, принадлежащих только ко двум классам. 

В первом графе датасета у нас имеется 7 узлов с 7 аттрибутами (features), и 38 рёбер с 4 аттрибутами каждое. Также у него имеется только 1 метка класса (`y=[1]`). 

In [3]:
torch.manual_seed(42)
dataset = dataset.shuffle()

train_dataset = dataset[:150]
test_dataset = dataset[150:]

print(f'Количество графов в тренировочной выборке: {len(train_dataset)}')
print(f'Количество графов в тестовой выборке: {len(test_dataset)}')

Количество графов в тренировочной выборке: 150
Количество графов в тестовой выборке: 38


Поскольку графы в данном наборе по своим размерам невелики, было бы правильным решением сформировать batch из них, перед тем как обучать GNN, чтобы использовать GPU полностью. В задачах обработки изображений или текста, батчи формируются изменяя форму матрицы (rescaling) или добавляя отступ (padding), чтобы привести каждый пример к одинаковой форме, а затем добавляется дополнительное измерение, где эти примеры группируются. Длина этого измерения равняется количеству примеров в батче и обычно обозначается как `batch_size`. 

Однако для графов эти подходы не подойдут, или подойдут, но приведут к чрезмерному использованию памяти. Поэтому, PyTorch Geometric использует другой подход. В этом подходе матрицы смежости (adjacency matrix) группируются по диагонали, как бы создавая один большой граф, состоящий из нескольких подграфов, а признаки узлов и целевые метки просто конкатенируются по размерности узлов. 

![batch_forming](batch_forming.png)

У этой процедуры есть 2 основных достоинства: 
1. Операторы GNN, основанные на message passing не требуют модификации, поскольку сообщения не передаются между узлами, принадлежащим разным графам.
2. Отсутствуют лишние затраты по памяти, поскольку матрицы смежности (adjacency matrix) хранятся в разреженном формате, содержащем только ненулевые элементы.

In [7]:
from torch_geometric.loader import DataLoader

train_loader = DataLoader(train_dataset, batch_size=64, shuffle = True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle = False)

for step, data in enumerate(train_loader):
    print(f'Шаг {step + 1}')
    print('-----------------------------')
    print(f'Количество графов в данном батче: {data.num_graphs}')
    print(data)
    print()


Шаг 1
-----------------------------
Количество графов в данном батче: 64
DataBatch(edge_index=[2, 2462], x=[1125, 7], edge_attr=[2462, 4], y=[64], batch=[1125], ptr=[65])

Шаг 2
-----------------------------
Количество графов в данном батче: 64
DataBatch(edge_index=[2, 2664], x=[1194, 7], edge_attr=[2664, 4], y=[64], batch=[1194], ptr=[65])

Шаг 3
-----------------------------
Количество графов в данном батче: 22
DataBatch(edge_index=[2, 850], x=[384, 7], edge_attr=[850, 4], y=[22], batch=[384], ptr=[23])



Получили 3 батча, 2 по 64 графа и один 22 (итого 150, вся выборка). Каждый `batch` объект сопровождается `batch` вектором, который сопоставляет каждому узлу его граф в батче.

# Обучение графовой нейронной сети (GNN)



1. Закодировать узел 